# **Simulatie alleen parafine**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### **Constanten**

In [2]:
dt = 0.004
dx = 0.00004
Tm = 54 + 273.15

l_l, rho_l, c_l = (0.15, 780, 2100)
l_s, rho_s, c_s = (0.24, 860, 2900)
L = 2.1e5 # J/kg

# x_staaf = 0.3 # (m) Staaf lengte
x = 0.1 #(m) Simluatie lengte
t = 1600 # (s) Simulatie tijd

T_0 = 35 + 273.15
T_x0 = 100 + 273.15

i_max, k_max = (int(x/dx), int(t/dt))

print('i_max: {}, k_max: {}'.format(i_max, k_max))

i_max: 2500, k_max: 400000


### **Variabelen die een functie van T zijn**

In [3]:
def theta_l(T, dT):
    if T >= Tm + dT:
        return 1
    elif Tm - dT < T and T < Tm + dT:
        return (T - Tm + dT)/2/dT
    else:
        return 0

def theta_s(T, dT):
    return 1 - theta_l(T, dT)

def l_phi(T, dT):
    return theta_l(T, dT) * l_l + theta_s(T, dT) * l_s

def rho_phi(T, dT):
    return theta_l(T, dT) * rho_l + theta_s(T, dT) * rho_s

def dthetal_dT(T, dT):
    if Tm - dT < T and T < Tm + dT:
        return 1/2/dT
    else:
        return 0

def cA(T, dT):
    T_ref = Tm - dT
    
    return theta_s(T, dT)*rho_s*c_s + theta_l(T, dT)*rho_l*c_l + ((rho_l*c_l - rho_s*c_s)*(T - T_ref) + rho_l*L) * dthetal_dT(T, dT)

### **Oplossen**

In [4]:
def calculate_T(dT):
    T = np.empty((k_max, i_max))
    T.fill(T_0)
    T[:,:1] = T_x0
    
    try:
        for k in tqdm(range(0, k_max-1)):
            for i in range(1, i_max-1):

                T[k + 1, i] = T[k,i] + (dt/dx**2/cA(T[k,i], dT))*((l_phi(T[k,i+1], dT)-l_phi(T[k,i], dT))*(T[k,i+1]-T[k,i])+l_phi(T[k,i], dT)*(T[k,i+1]-2*T[k,i]+T[k,i-1]))
                
    except Exception as e:
        print(e)
        return T
    return T

dT_ar = [3,2,1,0.5,0.25]


for dT in dT_ar:
    T_grid = calculate_T(dT)
    # np.savetxt('opgeslagen_np_arrays/T-dt-{}-dx-{}-dT-{}.txt'.format(dt, dx, dT), T_grid, fmt='%d')

100%|██████████| 399999/399999 [1:37:25<00:00, 68.43it/s]  


In [6]:
# print('Smeltfront positie als functie van de tijd voor verschillende dT')

def calc_melt_height(T):
    melt_height = []
    melt_time = []
    for idx, k in enumerate(T):
        pos = np.argmin(np.array(k)>Tm)
        melt_height.append(pos*dx)
        melt_time.append(idx*dt)
        
    return melt_height, melt_time

plt.figure()

for dT in dT_ar:
    T_grid = np.loadtxt('opgeslagen_np_arrays/T-dt-{}-dx-{}-dT-{}.txt'.format(dt, dx, dT), dtype=int)
    melt_height, melt_time = calc_melt_height(T_grid)

    plt.plot(melt_time,melt_height, label='dT: {}'.format(dT))
    
plt.xlabel('t (s)')
plt.ylabel('h (m)')
plt.legend()
plt.savefig('figs/smeltfront-dT.eps')
plt.show()